In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

# Set page config
st.set_page_config(
    page_title="RIL Stock Price Forecaster",
    page_icon="📈",
    layout="wide"
)

# Custom CSS
st.markdown("""
<style>
    .main-header {
        background: linear-gradient(135deg, #3b82f6 0%, #6366f1 100%);
        padding: 2rem;
        border-radius: 10px;
        color: white;
        margin-bottom: 2rem;
    }
    .metric-card {
        background: white;
        padding: 1rem;
        border-radius: 10px;
        border-left: 4px solid #3b82f6;
        box-shadow: 0 2px 4px rgba(0,0,0,0.1);
        transition: transform 0.2s;
    }
    .metric-card:hover {
        transform: translateY(-2px);
    }
    .model-btn {
        padding: 0.5rem 1rem;
        border-radius: 8px;
        border: 2px solid #e5e7eb;
        background: white;
        color: #374151;
        font-weight: 500;
        cursor: pointer;
        transition: all 0.3s;
    }
    .model-btn:hover {
        background: #f9fafb;
    }
    .model-btn.active {
        background: #3b82f6;
        color: white;
        border-color: #3b82f6;
        transform: scale(1.05);
        box-shadow: 0 4px 6px rgba(59, 130, 246, 0.5);
    }
</style>
""", unsafe_allow_html=True)

def generate_historical_data():
    """Generate historical data from Dec 2023 to Dec 2025"""
    dates = pd.date_range(start='2023-12-01', end='2025-12-02', freq='D')
    base_price = 1250
    prices = []

    for i, date in enumerate(dates):
        # Simulate price movement with trend and seasonality
        trend = i * 0.3
        seasonality = 50 * np.sin(i / 30)
        noise = (np.random.random() - 0.5) * 30
        jv_boost = 40 if i > 700 else 0  # Meta JV impact

        price = base_price + trend + seasonality + noise + jv_boost
        prices.append(round(price, 2))

    return pd.DataFrame({
        'date': dates,
        'price': prices,
        'type': 'historical'
    })

def arima_forecast(historical_df):
    """ARIMA-like forecast"""
    last_price = historical_df['price'].iloc[-1]
    forecast_dates = pd.date_range(start='2026-01-01', periods=365, freq='D')
    prices = []

    momentum = 0.15  # Positive momentum from JV

    for i, date in enumerate(forecast_dates):
        trend = i * momentum
        seasonality = 30 * np.sin(i / 40)
        dampening = np.exp(-i / 500)

        price = last_price + trend + seasonality * dampening
        prices.append({
            'date': date,
            'price': round(price, 2),
            'type': 'arima'
        })

    return pd.DataFrame(prices)

def exp_smoothing_forecast(historical_df):
    """Exponential smoothing forecast"""
    last_price = historical_df['price'].iloc[-1]
    forecast_dates = pd.date_range(start='2026-01-01', periods=365, freq='D')
    prices = []

    alpha = 0.3
    beta = 0.1
    level = last_price
    trend_val = 0.2

    for i, date in enumerate(forecast_dates):
        level = level + trend_val
        trend_val = beta * trend_val + (1 - beta) * 0.18
        seasonal = 25 * np.sin(i / 35)

        price = level + seasonal
        prices.append({
            'date': date,
            'price': round(price, 2),
            'type': 'exp_smoothing'
        })

    return pd.DataFrame(prices)

def prophet_forecast(historical_df):
    """Prophet-like forecast with events"""
    last_price = historical_df['price'].iloc[-1]
    forecast_dates = pd.date_range(start='2026-01-01', periods=365, freq='D')
    prices = []

    growth_rate = 0.0004  # Daily growth from AI JV

    for i, date in enumerate(forecast_dates):
        # Base trend with compounding
        base_trend = last_price * np.power(1 + growth_rate, i)

        # Seasonal components
        yearly_pattern = 40 * np.sin((i / 365) * 2 * np.pi)
        quarterly_pattern = 20 * np.sin((i / 90) * 2 * np.pi)

        # Event impact (quarterly earnings, JV milestones)
        event_impact = 0
        if i in [90, 180, 270]:
            event_impact = 25  # Earnings
        if i == 120:
            event_impact = 40  # Major JV milestone

        price = base_trend + yearly_pattern + quarterly_pattern + event_impact
        prices.append({
            'date': date,
            'price': round(price, 2),
            'type': 'prophet'
        })

    return pd.DataFrame(prices)

def lstm_forecast(historical_df):
    """LSTM-inspired forecast (simulated deep learning)"""
    recent = historical_df['price'].tail(90)
    avg_price = recent.mean()
    momentum = recent.iloc[-1] - recent.iloc[0]

    forecast_dates = pd.date_range(start='2026-01-01', periods=365, freq='D')
    prices = []
    price = recent.iloc[-1]

    for i, date in enumerate(forecast_dates):
        # LSTM learns patterns
        trend_component = momentum / 90 * 1.2
        volatility = 15 * np.sin(i / 20) * np.exp(-i / 400)
        long_term_growth = i * 0.16

        price = avg_price + trend_component * i + volatility + long_term_growth
        prices.append({
            'date': date,
            'price': round(price, 2),
            'type': 'lstm'
        })

    return pd.DataFrame(prices)

def ensemble_forecast(historical_df):
    """Ensemble forecast combining all models"""
    arima_df = arima_forecast(historical_df)
    exp_smooth_df = exp_smoothing_forecast(historical_df)
    prophet_df = prophet_forecast(historical_df)
    lstm_df = lstm_forecast(historical_df)

    # Define weights
    weights = {'arima': 0.2, 'exp_smoothing': 0.2, 'prophet': 0.35, 'lstm': 0.25}

    ensemble_data = []
    for i in range(len(arima_df)):
        date = arima_df['date'].iloc[i]

        ensemble_price = (
            arima_df['price'].iloc[i] * weights['arima'] +
            exp_smooth_df['price'].iloc[i] * weights['exp_smoothing'] +
            prophet_df['price'].iloc[i] * weights['prophet'] +
            lstm_df['price'].iloc[i] * weights['lstm']
        )

        # Calculate confidence intervals
        prices = [
            arima_df['price'].iloc[i],
            exp_smooth_df['price'].iloc[i],
            prophet_df['price'].iloc[i],
            lstm_df['price'].iloc[i]
        ]
        std = np.std(prices)

        ensemble_data.append({
            'date': date,
            'price': round(ensemble_price, 2),
            'lower': round(ensemble_price - 1.96 * std, 2),
            'upper': round(ensemble_price + 1.96 * std, 2),
            'type': 'ensemble'
        })

    return pd.DataFrame(ensemble_data)

def calculate_metrics(forecast_df, current_price):
    """Calculate forecast metrics"""
    year_2026 = forecast_df[forecast_df['date'].dt.year == 2026]

    avg_price = year_2026['price'].mean()
    end_price = year_2026['price'].iloc[-1]
    expected_return = ((end_price - current_price) / current_price * 100)
    high_price = year_2026['price'].max()
    low_price = year_2026['price'].min()

    return {
        'current': round(current_price, 2),
        'end_year': round(end_price, 2),
        'average': round(avg_price, 2),
        'expected_return': round(expected_return, 2),
        'high': round(high_price, 2),
        'low': round(low_price, 2)
    }

def create_chart(historical_df, forecast_df, model_type):
    """Create interactive Plotly chart"""
    # Combine historical and forecast data
    combined_df = pd.concat([
        historical_df.tail(180),
        forecast_df
    ])

    # Create figure
    fig = go.Figure()

    # Historical data
    hist_mask = combined_df['type'] == 'historical'
    fig.add_trace(go.Scatter(
        x=combined_df[hist_mask]['date'],
        y=combined_df[hist_mask]['price'],
        mode='lines',
        name='Historical',
        line=dict(color='#3b82f6', width=3),
        fill='tozeroy',
        fillcolor='rgba(59, 130, 246, 0.1)'
    ))

    # Forecast data
    forecast_mask = combined_df['type'] != 'historical'
    fig.add_trace(go.Scatter(
        x=combined_df[forecast_mask]['date'],
        y=combined_df[forecast_mask]['price'],
        mode='lines',
        name='Forecast',
        line=dict(color='#10b981', width=3),
        fill='tozeroy',
        fillcolor='rgba(16, 185, 129, 0.1)'
    ))

    # Confidence intervals for ensemble model
    if model_type == 'ensemble' and 'lower' in forecast_df.columns:
        fig.add_trace(go.Scatter(
            x=pd.concat([forecast_df['date'], forecast_df['date'][::-1]]),
            y=pd.concat([forecast_df['upper'], forecast_df['lower'][::-1]]),
            fill='toself',
            fillcolor='rgba(16, 185, 129, 0.2)',
            line=dict(color='rgba(255,255,255,0)'),
            name='95% Confidence',
            showlegend=True
        ))

    # Update layout
    fig.update_layout(
        title='RIL Stock Price Forecast',
        xaxis_title='Date',
        yaxis_title='Price (₹)',
        hovermode='x unified',
        template='plotly_white',
        height=500,
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01
        ),
        xaxis=dict(
            tickformat='%m/%y'
        ),
        yaxis=dict(
            tickprefix='₹'
        )
    )

    return fig

def main():
    # Header
    st.markdown("""
    <div class="main-header">
        <h1 style="font-size: 2.5rem; margin-bottom: 0.5rem;">RIL Stock Price Forecaster</h1>
        <p style="font-size: 1.2rem; opacity: 0.9;">Advanced Time Series Prediction for 2026</p>
        <p style="font-size: 0.9rem; opacity: 0.8;">Incorporating Meta JV Impact & AI Growth</p>
    </div>
    """, unsafe_allow_html=True)

    # Initialize session state for selected model
    if 'selected_model' not in st.session_state:
        st.session_state.selected_model = 'ensemble'

    # Model selection
    st.subheader("Select Forecasting Model")

    models = [
        {'id': 'ensemble', 'name': 'Ensemble (Best)', 'color': 'blue'},
        {'id': 'prophet', 'name': 'Prophet', 'color': 'green'},
        {'id': 'lstm', 'name': 'LSTM', 'color': 'purple'},
        {'id': 'arima', 'name': 'ARIMA', 'color': 'orange'},
        {'id': 'exp_smoothing', 'name': 'Exp Smoothing', 'color': 'pink'}
    ]

    # Create columns for model buttons
    cols = st.columns(5)
    for i, model in enumerate(models):
        with cols[i]:
            is_active = st.session_state.selected_model == model['id']
            btn_class = "model-btn active" if is_active else "model-btn"

            st.markdown(f"""
            <div class="{btn_class}" onclick="window.parent.postMessage({{'type': 'streamlit:setComponentValue', 'key': 'model_{model['id']}', 'value': '{model['id']}'}}, '*')">
                {model['name']}
            </div>
            """, unsafe_allow_html=True)

            # Add click handler
            if st.button(model['name'], key=f"model_{model['id']}"):
                st.session_state.selected_model = model['id']
                st.rerun()

    # Generate data
    historical_df = generate_historical_data()
    current_price = historical_df['price'].iloc[-1]

    # Get forecast based on selected model
    with st.spinner(f'Running {st.session_state.selected_model} forecast...'):
        if st.session_state.selected_model == 'arima':
            forecast_df = arima_forecast(historical_df)
        elif st.session_state.selected_model == 'exp_smoothing':
            forecast_df = exp_smoothing_forecast(historical_df)
        elif st.session_state.selected_model == 'prophet':
            forecast_df = prophet_forecast(historical_df)
        elif st.session_state.selected_model == 'lstm':
            forecast_df = lstm_forecast(historical_df)
        else:
            forecast_df = ensemble_forecast(historical_df)

    # Calculate metrics
    metrics = calculate_metrics(forecast_df, current_price)

    # Display metrics
    st.subheader("Forecast Metrics")

    metrics_cols = st.columns(6)

    metric_configs = [
        {'label': 'Current Price', 'value': f"₹{metrics['current']}", 'icon': '💰'},
        {'label': 'End 2026', 'value': f"₹{metrics['end_year']}", 'icon': '📅'},
        {'label': 'Avg 2026', 'value': f"₹{metrics['average']}", 'icon': '📊'},
        {'label': 'Expected Return', 'value': f"{metrics['expected_return']}%", 'icon': '📈'},
        {'label': '2026 High', 'value': f"₹{metrics['high']}", 'icon': '🚀'},
        {'label': '2026 Low', 'value': f"₹{metrics['low']}", 'icon': '⚠️'}
    ]

    for i, (col, metric) in enumerate(zip(metrics_cols, metric_configs)):
        with col:
            color = 'red' if i == 5 else ('green' if i == 3 and metrics['expected_return'] > 0 else 'blue')
            st.markdown(f"""
            <div class="metric-card" style="border-left-color: {color};">
                <div style="display: flex; align-items: center; margin-bottom: 0.5rem; color: {color};">
                    <span style="font-size: 1.2rem; margin-right: 0.5rem;">{metric['icon']}</span>
                    <span style="font-size: 0.8rem; font-weight: 600; text-transform: uppercase;">{metric['label']}</span>
                </div>
                <div style="font-size: 1.5rem; font-weight: 700; color: {color};">{metric['value']}</div>
            </div>
            """, unsafe_allow_html=True)

    # Chart
    st.subheader("Price Forecast Chart")
    fig = create_chart(historical_df, forecast_df, st.session_state.selected_model)
    st.plotly_chart(fig, use_container_width=True)

    # Analysis section
    st.subheader("Forecast Analysis & Key Drivers")

    col1, col2 = st.columns(2)

    with col1:
        st.markdown("""
        <div style="background: white; padding: 1rem; border-radius: 10px; box-shadow: 0 2px 4px rgba(0,0,0,0.1);">
            <h4 style="color: #374151; margin-bottom: 1rem;">✅ Positive Catalysts</h4>
            <ul style="color: #6b7280; line-height: 1.8;">
                <li>Meta JV (70% stake) driving AI revenue growth</li>
                <li>Enterprise AI adoption across Indian businesses</li>
                <li>Leveraging existing digital infrastructure</li>
                <li>Analyst targets: ₹1,680-₹1,785 (14% upside)</li>
                <li>Strong core businesses (Oil, Retail, Telecom)</li>
            </ul>
        </div>
        """, unsafe_allow_html=True)

    with col2:
        st.markdown("""
        <div style="background: white; padding: 1rem; border-radius: 10px; box-shadow: 0 2px 4px rgba(0,0,0,0.1);">
            <h4 style="color: #374151; margin-bottom: 1rem;">⚠️ Risk Factors</h4>
            <ul style="color: #6b7280; line-height: 1.8;">
                <li>JV execution and scaling challenges</li>
                <li>Enterprise AI adoption timeline uncertainty</li>
                <li>Regulatory changes in AI/data governance</li>
                <li>Macro economic conditions & market volatility</li>
                <li>Competition in AI/cloud infrastructure space</li>
            </ul>
        </div>
        """, unsafe_allow_html=True)

    # Model note
    st.markdown("""
    <div style="background: #dbeafe; padding: 1rem; border-radius: 10px; border: 1px solid #93c5fd; margin-top: 2rem;">
        <p style="color: #374151; font-size: 0.9rem;">
            <strong>📝 Model Note:</strong> The {} forecast RIL prices based on historical patterns, seasonal trends, and the anticipated impact of the Meta JV. The ensemble approach provides confidence intervals (95%) to account for uncertainty. Actual results may vary based on market conditions and execution.
        </p>
    </div>
    """.format(
        'ensemble model combines ARIMA, Prophet, LSTM, and Exponential Smoothing'
        if st.session_state.selected_model == 'ensemble'
        else f'{st.session_state.selected_model.upper()} model'
    ), unsafe_allow_html=True)

if __name__ == "__main__":
    main()